## Set up the experiment folder

In [1]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'driver-training'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('../data/porto_seguro_safe_driver_prediction_input.csv', os.path.join(training_folder, "porto_seguro_safe_driver_prediction_input.csv"))


'driver-training/porto_seguro_safe_driver_prediction_input.csv'

## train.py
This file defines the key functions required to train the model.  
The file can be invoked with `python train.py` for development purposes.

In [2]:
%%writefile $training_folder/train.py
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
import lightgbm


def split_data(data_df):
    """Split a dataframe into training and validation datasets"""

    features = data_df.drop(['target', 'id'], axis=1)
    labels = np.array(data_df['target'])
    features_train, features_valid, labels_train, labels_valid = train_test_split(
        features, labels, test_size=0.2, random_state=0)

    train_data = lightgbm.Dataset(features_train, label=labels_train)
    valid_data = lightgbm.Dataset(
        features_valid, label=labels_valid, free_raw_data=False)

    return (train_data, valid_data)


def train_model(data, parameters):
    """Train a model with the given datasets and parameters"""
    # The object returned by split_data is a tuple.
    # Access train_data with data[0] and valid_data with data[1]

    train_data = data[0]
    valid_data = data[1]
    model = lightgbm.train(parameters,
                           train_data,
                           valid_sets=valid_data,
                           num_boost_round=500,
                           early_stopping_rounds=20)

    return model


def get_model_metrics(model, data):
    """Construct a dictionary of metrics for the model"""

    valid_data = data[1]
    predictions = model.predict(valid_data.data)
    fpr, tpr, thresholds = metrics.roc_curve(valid_data.label, predictions)
    model_metrics = {"auc": (metrics.auc(fpr, tpr))}

    return model_metrics


Overwriting driver-training/train.py


## parameters.json
This file will specify the parameters used to train the model.

In [3]:
%%writefile $training_folder/parameters.json
{
    "training":
    {
        "learning_rate": 0.02,
        "boosting_type": "gbdt",
        "objective": "binary",
        "metric": "auc",
        "sub_feature": 0.7,
        "num_leaves": 60,
        "min_data": 100,
        "min_hessian": 1,
        "verbose": 0
    }
}


Overwriting driver-training/parameters.json


## driver_training.py
This file will be the entry script when running an Azure ML context.  
It calls the functions defined in train.py for data preparation and training, but reads parameters from a file, and logs output to the Azure ML context.  
The file can be invoked with `python driver_training.py` for development purposes.

In [4]:
%%writefile $training_folder/driver_training.py
# Import libraries
import argparse
from azureml.core import Run
import joblib
import json
import os
import pandas as pd
import shutil

# Import functions from train.py
from train import split_data, train_model, get_model_metrics

# Get the output folder for the model from the '--output_folder' parameter
parser = argparse.ArgumentParser()
parser.add_argument('--output_folder', type=str, dest='output_folder', default="outputs")
args = parser.parse_args()
output_folder = args.output_folder

# Get the experiment run context
run = Run.get_context()

# load the safe driver prediction dataset
train_df = pd.read_csv('porto_seguro_safe_driver_prediction_input.csv')

# Load the parameters for training the model from the file
with open("parameters.json") as f:
    pars = json.load(f)
    parameters = pars["training"]

# Log each of the parameters to the run
for param_name, param_value in parameters.items():
    run.log(param_name, param_value)
    
# Use the functions imported from train.py to prepare data, train the model, and calculate the metrics
data = split_data(train_df)
model = train_model(data, parameters)
model_metrics = get_model_metrics(model, data)
run.log('auc', model_metrics['auc'])

# Save the trained model to the output folder
os.makedirs(output_folder, exist_ok=True)
output_path = output_folder + "/porto_seguro_safe_driver_model.pkl"
joblib.dump(value=model, filename=output_path)

run.complete()

Overwriting driver-training/driver_training.py


In [5]:
import azureml.core
from azureml.core import Workspace

# Load the workspace
ws = Workspace.from_config()

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (portalocker 2.0.0 (/anaconda/envs/azureml_py36/lib/python3.6/site-packages), Requirement.parse('portalocker~=1.0'), {'msal-extensions'}).


## Use an Estimator to Run the Script as an Experiment

See [this tutorial](https://github.com/MicrosoftDocs/mslearn-aml-labs/blob/master/02-Training_Models.ipynb) for a starting point

Use the scikit-learn and lightgbm conda packages

In [6]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment

# Create an estimator
estimator = Estimator(source_directory='driver-training',
                      entry_script='driver_training.py',
                      compute_target='local',
                      conda_packages=['scikit-learn', 'lightgbm']
                      )

# Create an experiment
experiment_name = 'driver-training'
experiment = Experiment(workspace = ws, name = experiment_name)

# Run the experiment based on the estimator
run = experiment.submit(config=estimator)
run.wait_for_completion(show_output=True)

RunId: driver-training_1598645501_c33c17ab
Web View: https://ml.azure.com/experiments/driver-training/runs/driver-training_1598645501_c33c17ab?wsid=/subscriptions/7c9d382c-5964-48db-9cf6-c595c7ba4339/resourcegroups/mlops-rg/workspaces/mlops-aml

Streaming azureml-logs/60_control_log.txt

[2020-08-28T20:11:48.021931] Using urllib.request Python 3.0 or later
Streaming log file azureml-logs/60_control_log.txt
Starting the daemon thread to refresh tokens in background for process with pid = 55909
Running: ['/bin/bash', '/tmp/azureml_runs/driver-training_1598645501_c33c17ab/azureml-environment-setup/docker_env_checker.sh']

Found materialized image on target: azureml/azureml_39f1c006542b3acd3a960ee0cfe79395


Logging experiment running status in history service.
Running: ['docker', 'run', '--name', 'driver-training_1598645501_c33c17ab', '--rm', '-v', '/tmp/azureml_runs/driver-training_1598645501_c33c17ab:/azureml-run', '--shm-size', '2g', '-e', 'EXAMPLE_ENV_VAR=EXAMPLE_VALUE', '-e', 'AZUREM

{'runId': 'driver-training_1598645501_c33c17ab',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-08-28T20:11:48.585683Z',
 'endTimeUtc': '2020-08-28T20:12:20.498289Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '521eb234-199f-436e-b6dc-1e443147f4dd',
  'azureml.git.repository_uri': 'https://github.com/kawo123/auzre-mlops.git',
  'mlflow.source.git.repoURL': 'https://github.com/kawo123/auzre-mlops.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': '9a32eae324e90c92cdefe87883bf5a15da72b275',
  'mlflow.source.git.commit': '9a32eae324e90c92cdefe87883bf5a15da72b275',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'runDefinition': {'script': 'driver_training.py',
  'scriptType': None,
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'o

In [7]:
# Print the resulting metrics
metrics = run.get_metrics()
print(metrics)
for k, v in metrics.items():
    print(k, v)

{'learning_rate': 0.02, 'boosting_type': 'gbdt', 'metric': 'auc', 'objective': 'binary', 'num_leaves': 60, 'min_data': 100, 'min_hessian': 1, 'sub_feature': 0.7, 'verbose': 0, 'auc': 0.6377511613946426}
learning_rate 0.02
boosting_type gbdt
metric auc
objective binary
num_leaves 60
min_data 100
min_hessian 1
sub_feature 0.7
verbose 0
auc 0.6377511613946426


In [9]:
# Register the model
run.register_model(model_path='outputs/porto_seguro_safe_driver_model.pkl', model_name='driver_model.pkl', tags={'auc': metrics['auc']})

Model(workspace=Workspace.create(name='mlops-aml', subscription_id='7c9d382c-5964-48db-9cf6-c595c7ba4339', resource_group='mlops-rg'), name=driver_model.pkl, id=driver_model.pkl:2, version=2, tags={'auc': '0.6377511613946426'}, properties={})